## The python code for Verifying the accuracy of our DLA

In [7]:
from pynq import Overlay
from pynq import MMIO
import time
design = Overlay("./Ultimate_70M/cnn.bit")

cdma_address = design.ip_dict['axi_cdma_0']['phys_addr']
axi_gpio_address0 = design.ip_dict['axi_gpio_0']['phys_addr'] # start
axi_gpio_address1 = design.ip_dict['axi_gpio_1']['phys_addr'] # done
axi_gpio_address2 = design.ip_dict['axi_gpio_2']['phys_addr'] # mode
axi_gpio_address3 = design.ip_dict['axi_gpio_3']['phys_addr'] # result

zynq_addr = 0x30000000


bram0_addr = 0xC0000000 # if1
bram1_addr = 0xC2000000 # if2
bram2_addr = 0xC4000000 # w1
bram3_addr = 0xC6000000 # w2
bram4_addr = 0xC8000000 # w3
bram5_addr = 0xCA000000 # w4
bram6_addr = 0xCC000000 # w5

zynq_sys = MMIO(zynq_addr,    0xC000)
cdma     = MMIO(cdma_address, 0xC000)

list_label = [20,8,25,8,4,6,15,16,26,23,25,26,2,13,8,4,6,11,15,8,14,7,7,26,6,23,14,25,2,20,7,20,26,6,26,2,8,2,26,8,26,21,24,8,2,7,6,23,23,8,2,14,6,2,26,23,8,8,11,1,17,8,11,16,11,24,8,8,21,26,7,4,4,25,2,8,26,14,8,7,4,2,11,17,2,11,8,7,26,17,8,5,8,4,2,6,8,14,6,6,11,26
]

'''

initial input file to bram0
    
    
'''
print('start initial!\n')


'''

initial conv1 weight to bram2
    
    
'''
print('Weight1 initial!\n')

bram2_offset = 0x0

with open("letter_conv1_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram2_offset,int('0x'+line1,16))      
        bram2_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram2_addr)
cdma.write(0x28,0x98) # 152 = 38 * 4

'''

initial conv2 weight to bram3
    
    
'''
print('Weight2 initial!\n')

bram3_offset = 0x0

with open("letter_conv2_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram3_offset,int('0x'+line1,16))      
        bram3_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram3_addr)
cdma.write(0x28,0x960) # 2400 = 600 * 4


'''

initial conv3 weight to bram4
    
    
'''
print('Weight3 initial!\n')

bram4_offset = 0x0

with open("letter_conv3_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram4_offset,int('0x'+line1,16))      
        bram4_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram4_addr)
cdma.write(0x28,0xBB80) # 48000 = 12000 * 4

'''

initial fc1 weight to bram5
    
    
'''
print('Weight4 initial!\n')

bram5_offset = 0x0

with open("letter_fc1_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram5_offset,int('0x'+line1,16))      
        bram5_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram5_addr)
cdma.write(0x28,0x2760) # 10080 = 2520 * 4

'''

initial fc2 weight to bram6
    
    
'''
print('Weight5 initial!\n')

bram6_offset = 0x0

with open("letter_fc2_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram6_offset,int('0x'+line1,16))      
        bram6_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram6_addr)
cdma.write(0x28,0x8DC) # 2268 = 567 * 4

print('initial finish!\n')


'''

Input start signal
 
''' 
gpio_a = MMIO(axi_gpio_address0,8)
gpio_b = MMIO(axi_gpio_address1,8)
gpio_c = MMIO(axi_gpio_address2,8)
gpio_d = MMIO(axi_gpio_address3,8)
gpio_a.write(0x4,0)
gpio_c.write(0x4,0)

total_time = 0
correct = 0
for i in range(102):
    f_if = open("letter_102_test.hex", "r")
    print('%dth Input feature map initial!\n' %(i+1))
    count = 0
    cc = 0
    lll = []
    bram0_offset = 0x0
    for line1 in f_if:
        count += 1
        if count >= (256*i+1):
            cc += 1
            if cc == 257:
                break  
            zynq_sys.write(bram0_offset,int('0x'+line1,16))      
            lll.append(line1)
            bram0_offset += 4
        '''
        if count >= (256*i+1):
            cc += 1
            if cc == 257:
                break  
        '''
        '''
        if count == 257:
            break
        '''  
    
    cdma.write(0x00,0x04) #start
    cdma.write(0x18,zynq_addr)
    cdma.write(0x20,bram0_addr)
    cdma.write(0x28,0x400) # 1024

    gpio_c.write(0,0) # write 1: number
    start_time = time.time()
    gpio_a.write(0,1) # write 1 to start
    gpio_a.write(0,0) # write 1 to start
    
    f_if.close()

    

    time.sleep(1)
    #print("--- %s seconds ---" % (time.time() - start_time))
    total_time += time.time() - start_time - 1
    if list_label[i] == gpio_d.read():
        print("[%d] Inference value = %d,  expected = %d, PASS!" %((i+1), gpio_d.read(), list_label[i]))
        correct += 1
    else:
        print("[%d] Inference value = %d,  expected = %d, FAIL!" %((i+1), gpio_d.read(), list_label[i]))
    #print("Inference value: ", gpio_d.read())
         

    



#if gpio_b.read() == 0xffffffff:


#print("Inference value: ", gpio_d.read())

print('All finish!\n')
print("PASS: %d, FAIL: %d" %(correct, 102-correct))
print("Accuracy = %f" %(correct/102))
print("Average time = %f" %(total_time/102))

start initial!

Weight1 initial!

Weight2 initial!

Weight3 initial!

Weight4 initial!

Weight5 initial!

initial finish!

1th Input feature map initial!

[1] Inference value = 6,  expected = 20, FAIL!
2th Input feature map initial!

[2] Inference value = 8,  expected = 8, PASS!
3th Input feature map initial!

[3] Inference value = 2,  expected = 25, FAIL!
4th Input feature map initial!

[4] Inference value = 8,  expected = 8, PASS!
5th Input feature map initial!

[5] Inference value = 4,  expected = 4, PASS!
6th Input feature map initial!

[6] Inference value = 6,  expected = 6, PASS!
7th Input feature map initial!

[7] Inference value = 15,  expected = 15, PASS!
8th Input feature map initial!

[8] Inference value = 16,  expected = 16, PASS!
9th Input feature map initial!

[9] Inference value = 26,  expected = 26, PASS!
10th Input feature map initial!

[10] Inference value = 23,  expected = 23, PASS!
11th Input feature map initial!

[11] Inference value = 25,  expected = 25, PASS!
12t

[101] Inference value = 11,  expected = 11, PASS!
102th Input feature map initial!

[102] Inference value = 26,  expected = 26, PASS!
All finish!

PASS: 95, FAIL: 7
Accuracy = 0.931373
Average time = 0.001223


In [5]:
from pynq import Overlay
from pynq import MMIO
import time
design = Overlay("./Ultimate_70M/cnn.bit")

cdma_address = design.ip_dict['axi_cdma_0']['phys_addr']
axi_gpio_address0 = design.ip_dict['axi_gpio_0']['phys_addr'] # start
axi_gpio_address1 = design.ip_dict['axi_gpio_1']['phys_addr'] # done
axi_gpio_address2 = design.ip_dict['axi_gpio_2']['phys_addr'] # mode
axi_gpio_address3 = design.ip_dict['axi_gpio_3']['phys_addr'] # result

zynq_addr = 0x30000000


bram0_addr = 0xC0000000 # if1
bram1_addr = 0xC2000000 # if2
bram2_addr = 0xC4000000 # w1
bram3_addr = 0xC6000000 # w2
bram4_addr = 0xC8000000 # w3
bram5_addr = 0xCA000000 # w4
bram6_addr = 0xCC000000 # w5

zynq_sys = MMIO(zynq_addr,    0xC000)
cdma     = MMIO(cdma_address, 0xC000)

list_label = [0,0,2,2,7,0,3,0,1,2,5,6,2,6,7,9,8,1,4,4,4,4,3,2,4,7,3,8,6,5,1,7,4,2,4,1,6,4,0,8,9,3,6,5,1,1,8,0,6,7,8,7,3,1,2,1,9,0,8,0,9,2,9,3,2,9,1,4,7,6,2,9,0,5,5,5,5,4,1,4,0,9,0,1,9,8,6,1,8,8,0,6,1,8,9,8,6,7,1,7,2,5,7,3,9,2,4,8,0,2,7,6,3,2,0,7,9,1,3,0,4,6,3,0,8,3,1,2,6,6,9,4,0,3,7,1,8,6,1,7,2,5,4,3,6,7,8,9,9,2,6,0,6,8,9,9,1,6,5,0,2,0,7,2,2,9,0,0,1,4,9,2,4,4,9,3,3,1,8,1,7,0,9,9,9,4,3,2,0,9,8,0,8,2,9,5,3,3,3,1,3,8,2,4,7,5,1,9,9,3,7,6,9,2,5,2,8,4,7,1,7,7,2,8,4,8,7,9,4,9,5,4,7,7,0,3,9,4,4,1,0,6,5,5,9
]

'''

initial input file to bram0
    
    
'''
print('start initial!\n')


'''

initial conv1 weight to bram2
    
    
'''
print('Weight1 initial!\n')

bram2_offset = 0x0

with open("number_conv1_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram2_offset,int('0x'+line1,16))      
        bram2_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram2_addr)
cdma.write(0x28,0x98) # 152 = 38 * 4

'''

initial conv2 weight to bram3
    
    
'''
print('Weight2 initial!\n')

bram3_offset = 0x0

with open("number_conv2_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram3_offset,int('0x'+line1,16))      
        bram3_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram3_addr)
cdma.write(0x28,0x960) # 2400 = 600 * 4


'''

initial conv3 weight to bram4
    
    
'''
print('Weight3 initial!\n')

bram4_offset = 0x0

with open("number_conv3_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram4_offset,int('0x'+line1,16))      
        bram4_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram4_addr)
cdma.write(0x28,0xBB80) # 48000 = 12000 * 4

'''

initial fc1 weight to bram5
    
    
'''
print('Weight4 initial!\n')

bram5_offset = 0x0

with open("number_fc1_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram5_offset,int('0x'+line1,16))      
        bram5_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram5_addr)
cdma.write(0x28,0x2760) # 10080 = 2520 * 4

'''

initial fc2 weight to bram6
    
    
'''
print('Weight5 initial!\n')

bram6_offset = 0x0

with open("number_fc2_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram6_offset,int('0x'+line1,16))      
        bram6_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram6_addr)
cdma.write(0x28,0x348) # 840 = 210 * 4

print('initial finish!\n')


'''

Input start signal
 
''' 
gpio_a = MMIO(axi_gpio_address0,8)
gpio_b = MMIO(axi_gpio_address1,8)
gpio_c = MMIO(axi_gpio_address2,8)
gpio_d = MMIO(axi_gpio_address3,8)
gpio_a.write(0x4,0)
gpio_c.write(0x4,0)

total_time = 0
correct = 0
for i in range(245):
    f_if = open("number_245_test.hex", "r")
    print('%dth Input feature map initial!\n' %(i+1))
    count = 0
    cc = 0
    lll = []
    bram0_offset = 0x0
    for line1 in f_if:
        count += 1
        if count >= (256*i+1):
            cc += 1
            if cc == 257:
                break  
            zynq_sys.write(bram0_offset,int('0x'+line1,16))      
            lll.append(line1)
            bram0_offset += 4
        '''
        if count >= (256*i+1):
            cc += 1
            if cc == 257:
                break  
        '''
        '''
        if count == 257:
            break
        '''  
    
    cdma.write(0x00,0x04) #start
    cdma.write(0x18,zynq_addr)
    cdma.write(0x20,bram0_addr)
    cdma.write(0x28,0x400) # 1024

    gpio_c.write(0,0) # write 1: number
    start_time = time.time()
    gpio_a.write(0,1) # write 1 to start
    gpio_a.write(0,0) # write 1 to start
    
    f_if.close()

    

    time.sleep(1)
    #print("--- %s seconds ---" % (time.time() - start_time))
    total_time += time.time() - start_time - 1
    if list_label[i] == gpio_d.read():
        print("[%d] Inference value = %d,  expected = %d, PASS!" %((i+1), gpio_d.read(), list_label[i]))
        correct += 1
    else:
        print("[%d] Inference value = %d,  expected = %d, FAIL!" %((i+1), gpio_d.read(), list_label[i]))
    #print("Inference value: ", gpio_d.read())
         

    



#if gpio_b.read() == 0xffffffff:


#print("Inference value: ", gpio_d.read())

print('All finish!\n')
print("PASS: %d, FAIL: %d" %(correct, 245-correct))
print("Accuracy = %f" %(correct/245))
print("Average time = %f" %(total_time/245))

start initial!

Weight1 initial!

Weight2 initial!

Weight3 initial!

Weight4 initial!

Weight5 initial!

initial finish!

1th Input feature map initial!

[1] Inference value = 0,  expected = 0, PASS!
2th Input feature map initial!

[2] Inference value = 0,  expected = 0, PASS!
3th Input feature map initial!

[3] Inference value = 2,  expected = 2, PASS!
4th Input feature map initial!

[4] Inference value = 2,  expected = 2, PASS!
5th Input feature map initial!

[5] Inference value = 7,  expected = 7, PASS!
6th Input feature map initial!

[6] Inference value = 0,  expected = 0, PASS!
7th Input feature map initial!

[7] Inference value = 3,  expected = 3, PASS!
8th Input feature map initial!

[8] Inference value = 0,  expected = 0, PASS!
9th Input feature map initial!

[9] Inference value = 1,  expected = 1, PASS!
10th Input feature map initial!

[10] Inference value = 2,  expected = 2, PASS!
11th Input feature map initial!

[11] Inference value = 5,  expected = 5, PASS!
12th Input feat

[102] Inference value = 5,  expected = 5, PASS!
103th Input feature map initial!

[103] Inference value = 7,  expected = 7, PASS!
104th Input feature map initial!

[104] Inference value = 3,  expected = 3, PASS!
105th Input feature map initial!

[105] Inference value = 9,  expected = 9, PASS!
106th Input feature map initial!

[106] Inference value = 2,  expected = 2, PASS!
107th Input feature map initial!

[107] Inference value = 4,  expected = 4, PASS!
108th Input feature map initial!

[108] Inference value = 8,  expected = 8, PASS!
109th Input feature map initial!

[109] Inference value = 0,  expected = 0, PASS!
110th Input feature map initial!

[110] Inference value = 2,  expected = 2, PASS!
111th Input feature map initial!

[111] Inference value = 7,  expected = 7, PASS!
112th Input feature map initial!

[112] Inference value = 6,  expected = 6, PASS!
113th Input feature map initial!

[113] Inference value = 3,  expected = 3, PASS!
114th Input feature map initial!

[114] Inference 

[202] Inference value = 8,  expected = 8, PASS!
203th Input feature map initial!

[203] Inference value = 2,  expected = 2, PASS!
204th Input feature map initial!

[204] Inference value = 4,  expected = 4, PASS!
205th Input feature map initial!

[205] Inference value = 7,  expected = 7, PASS!
206th Input feature map initial!

[206] Inference value = 5,  expected = 5, PASS!
207th Input feature map initial!

[207] Inference value = 1,  expected = 1, PASS!
208th Input feature map initial!

[208] Inference value = 9,  expected = 9, PASS!
209th Input feature map initial!

[209] Inference value = 9,  expected = 9, PASS!
210th Input feature map initial!

[210] Inference value = 3,  expected = 3, PASS!
211th Input feature map initial!

[211] Inference value = 7,  expected = 7, PASS!
212th Input feature map initial!

[212] Inference value = 5,  expected = 6, FAIL!
213th Input feature map initial!

[213] Inference value = 9,  expected = 9, PASS!
214th Input feature map initial!

[214] Inference 